## 🚀 Momentum Stock Screener Framework

### 📥 Setups

Installation and import of required packages

In [61]:
# !pip install -r requirements.txt

In [62]:
import yfinance as yf

import pandas as pd
import numpy as np

import os

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

Initialization of analysis parameters

In [63]:
TICKER: str = 'Nasdaq-100'
TABLE_NUM: int = 4  # For webscraping data
TABLE_COL: str = 'Ticker' # For webscraping data

START_DATE: str = '2019-01-01'
END_DATE: str = '2025-08-08'

### 🏗️ Data Acquisition

In [66]:
def fetch_stock_data(index: str=TICKER, table_num: int=TABLE_NUM, table_col: str=TABLE_COL, start_date: str=START_DATE, end_date: str=END_DATE) -> pd.DataFrame:
    """
    Fetch historical stock data for all tickers in the index, both saved as .csv and returned as a dictionary of DataFrames.

    Parameters:
        index (str): Wikipedia page name for the index (Default to TICKER).
        table_num (int): Wikipedia table number for the index (Default to TABLE_NUM).
        table_col (str): Wikipedia table column for the index (Default to TABLE_COL).
        start_date (str): Start date in 'YYYY-MM-DD' format (Default to START_DATE).
        end_date (str): End date in 'YYYY-MM-DD' format (Default to END_DATE).

    Returns:
        pd.DataFrame: OHLCV data for all tickers in the index.
    """
    url = f"https://en.wikipedia.org/wiki/{index}"

    try:
        ttable = pd.read_html(url)
    except Exception as e:
        raise ConnectionError(f"Failed to fetch tables from {url}: {e}")

    if table_num >= len(ttable) or table_col not in ttable[table_num].columns:
        raise ValueError(f"No table with recognizable ticker column found in {url}.")
    
    tickers = ttable[table_num][table_col].tolist()

    os.makedirs("data", exist_ok=True)
    data_dict = {}
    
    try:
        data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)
    except Exception as e:
        print(f"Error downloading")
    
    for ticker in tickers:
        df = data.xs(ticker, axis=1, level=1)
        df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

        if df.empty:
            print(f"Warning: No data for {ticker}")
        else:
            data_dict[ticker] = df

            file_path = os.path.join("data", f"{ticker}.csv")
            df.to_csv(file_path)

    return data_dict

### 📊 Stock Analysis

In [ ]:
def calculate_indicators():
    return